In [1]:
from sqlalchemy.engine import result
from sqlalchemy import create_engine
from sqlalchemy import MetaData, ForeignKey
from sqlalchemy import Table, Column, Integer, String, delete
import numpy as np
import pandas as pd
import pymysql

# Connecting To DB & Import DS

In [2]:
dataset = pd.read_csv('movie.csv')
meta = MetaData()

SERVER='localhost'
USERNAME='root'
PASSWORD='1051047595Ma'
database='schema_test'
engine = create_engine(f'mysql+pymysql://{USERNAME}:{PASSWORD}@{SERVER}:3306/{database}')
conn = engine.connect()

Dataset Info

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     250 non-null    object
 1   title        250 non-null    object
 2   year         250 non-null    int64 
 3   parental     250 non-null    object
 4   runtime      250 non-null    int64 
 5   genre        250 non-null    object
 6   id_director  250 non-null    object
 7   director     250 non-null    object
 8   ids_writer   250 non-null    object
 9   writers      250 non-null    object
 10  ids_stars    250 non-null    object
 11  stars        250 non-null    object
 12  gus          221 non-null    object
dtypes: int64(2), object(11)
memory usage: 25.5+ KB


In [31]:
dataset.head()

,movie_id,title,year,parental,runtime,genre,id_director,director,ids_writer,writers,ids_stars,stars,gus
0,00111161,The Shawshank Redemption,1994,R,142,['Drama'],['0001104'],['Frank Darabont'],"['0000175', '0001104']","['Stephen King', 'Frank Darabont']","['0000209', '0000151', '0348409']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",28767189
1,00068646,The Godfather,1972,R,175,"['Crime', 'Drama']",['0000338'],['Francis Ford Coppola'],"['0701374', '0000338']","['Mario Puzo', 'Francis Ford Coppola']","['0000008', '0000199', '0001001']","['Marlon Brando', 'Al Pacino', 'James Caan']",136381073
2,00468569,The Dark Knight,2008,PG-13,152,"['Action', 'Crime', 'Drama']",['0634240'],['Christopher Nolan'],"['0634300', '0634240', '0275286']","['Jonathan Nolan', 'Christopher Nolan', 'David...","['0000288', '0005132', '0001173']","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",534987076
3,00071562,The Godfather Part II,1974,R,202,"['Crime', 'Drama']",['0000338'],['Francis Ford Coppola'],"['0000338', '0701374']","['Francis Ford Coppola', 'Mario Puzo']","['0000199', '0000134', '0000380']","['Al Pacino', 'Robert De Niro', 'Robert Duvall']",47834595
4,00050083,12 Angry Men,1957,Approved,96,"['Crime', 'Drama']",['0001486'],['Sidney Lumet'],['0741627'],['Reginald Rose'],"['0000020', '0002011', '0000842']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam']",None


In [4]:
def charize(ids):
    ids = str(ids)
    diff = 8-len(ids)
    if diff > 0:
        ids = diff*'0' + ids
    return ids

In [5]:
dataset['movie_id'] = dataset['movie_id'].astype(object)
dataset['gus'] = dataset['gus'].astype('Int64')
dataset = dataset.replace({pd.NA: None})

dataset['movie_id'] = dataset['movie_id'].apply(charize)

Create & Insert movies Table

In [4]:
movies = Table(
    'movies', meta,
    Column('id', String(8), primary_key = True), 
    Column('title', String(128), nullable=False), 
    Column('year', Integer),
    Column('runtime', Integer),
    Column('parental_guide', String(8)),
    Column('gross_us_canda', Integer, nullable=True) 
)
# meta.create_all(engine)

In [33]:
conn1 = pymysql.connect(host='localhost', user='root', passwd='1051047595Ma', db='schema_test')

# Insert desired columns from dataframe into movies table
with conn1:
    cursor = conn1.cursor()
    for index, row in dataset[['movie_id', 'title', 'year', 'runtime', 'parental', 'gus']].iterrows():
        sql = "INSERT INTO movies (id, title, year, runtime, parental_guide, gross_us_canda) VALUES (%s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, tuple(row))
    conn1.commit()
    
print("DataFrame inserted into MySQL")

DataFrame inserted into MySQL


Customize Data

In [12]:
from ast import literal_eval
full_ds = dataset.copy()

full_ds['genre'] = full_ds['genre'].apply(literal_eval)
# full_ds['id_director'] = full_ds['id_director'].apply(literal_eval)
# full_ds['director'] = full_ds['director'].apply(literal_eval)
# full_ds['ids_writer'] = full_ds['ids_writer'].apply(literal_eval)
# full_ds['writers'] = full_ds['writers'].apply(literal_eval)
# full_ds['ids_stars'] = full_ds['ids_stars'].apply(literal_eval)
# full_ds['stars'] = full_ds['stars'].apply(literal_eval)
# Explode array columns into rows
full_ds =full_ds.explode('genre')
# full_ds =full_ds.explode('id_director')
# full_ds =full_ds.explode('director')
# full_wr =full_ds.explode('ids_writer')
# full_ds =full_ds.explode('ids_writer')
# full_ds = full_ds.explode('writers')

# full_ds =full_ds.explode('ids_stars')
# full_ds =full_ds.explode('stars')

# Reset index to remove old index  
# full_ds =full_ds.reset_index(drop=True)
full_ds.head(10)

,movie_id,title,year,parental,runtime,genre,id_director,director,ids_writer,writers,ids_stars,stars,gus
0,00111161,The Shawshank Redemption,1994,R,142,Drama,['0001104'],['Frank Darabont'],"['0000175', '0001104']","['Stephen King', 'Frank Darabont']","['0000209', '0000151', '0348409']","['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']",28767189
1,00068646,The Godfather,1972,R,175,Crime,['0000338'],['Francis Ford Coppola'],"['0701374', '0000338']","['Mario Puzo', 'Francis Ford Coppola']","['0000008', '0000199', '0001001']","['Marlon Brando', 'Al Pacino', 'James Caan']",136381073
1,00068646,The Godfather,1972,R,175,Drama,['0000338'],['Francis Ford Coppola'],"['0701374', '0000338']","['Mario Puzo', 'Francis Ford Coppola']","['0000008', '0000199', '0001001']","['Marlon Brando', 'Al Pacino', 'James Caan']",136381073
2,00468569,The Dark Knight,2008,PG-13,152,Action,['0634240'],['Christopher Nolan'],"['0634300', '0634240', '0275286']","['Jonathan Nolan', 'Christopher Nolan', 'David...","['0000288', '0005132', '0001173']","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",534987076
2,00468569,The Dark Knight,2008,PG-13,152,Crime,['0634240'],['Christopher Nolan'],"['0634300', '0634240', '0275286']","['Jonathan Nolan', 'Christopher Nolan', 'David...","['0000288', '0005132', '0001173']","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",534987076
2,00468569,The Dark Knight,2008,PG-13,152,Drama,['0634240'],['Christopher Nolan'],"['0634300', '0634240', '0275286']","['Jonathan Nolan', 'Christopher Nolan', 'David...","['0000288', '0005132', '0001173']","['Christian Bale', 'Heath Ledger', 'Aaron Eckh...",534987076
3,00071562,The Godfather Part II,1974,R,202,Crime,['0000338'],['Francis Ford Coppola'],"['0000338', '0701374']","['Francis Ford Coppola', 'Mario Puzo']","['0000199', '0000134', '0000380']","['Al Pacino', 'Robert De Niro', 'Robert Duvall']",47834595
3,00071562,The Godfather Part II,1974,R,202,Drama,['0000338'],['Francis Ford Coppola'],"['0000338', '0701374']","['Francis Ford Coppola', 'Mario Puzo']","['0000199', '0000134', '0000380']","['Al Pacino', 'Robert De Niro', 'Robert Duvall']",47834595
4,00050083,12 Angry Men,1957,Approved,96,Crime,['0001486'],['Sidney Lumet'],['0741627'],['Reginald Rose'],"['0000020', '0002011', '0000842']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam']",None
4,00050083,12 Angry Men,1957,Approved,96,Drama,['0001486'],['Sidney Lumet'],['0741627'],['Reginald Rose'],"['0000020', '0002011', '0000842']","['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam']",None


Person

In [5]:
id_pers = full_ds['id_director'].unique()
pers = full_ds['director'].unique()
id_pers = np.concatenate((id_pers ,full_ds['ids_writer'].unique(), full_ds['ids_stars'].unique()))
pers = np.concatenate((pers ,full_ds['writers'].unique(), full_ds['stars'].unique()))
person_df = pd.DataFrame({'id': id_pers,
                     'name': pers})

person_df.describe()

,id,name
count,1200,1200
unique,1076,1074
top,0000080,Orson Welles
freq,3,3


In [6]:
person_df.drop_duplicates(inplace=True)
person_df.describe()

,id,name
count,1076,1076
unique,1076,1074
top,0001104,Steve McQueen
freq,1,2


In [28]:
person = Table(
    'person', meta,
    Column('id', String(8), primary_key = True), 
    Column('name', String(32))
)
# meta.create_all(engine)

In [8]:
conn1 = pymysql.connect(host='localhost', user='root', passwd='1051047595Ma', db='schema_test')

# Insert desired columns from dataframe into movies table
with conn1:
    cursor = conn1.cursor()
    for index, row in person_df[['id', 'name']].iterrows():
        sql = "INSERT INTO person (id, name) VALUES (%s, %s)"
        cursor.execute(sql, tuple(row))
    conn1.commit()
    
print("DataFrame inserted into MySQL")

DataFrame inserted into MySQL


Cast

In [10]:
id_movie = full_ds['movie_id']
id_star = full_ds['ids_stars']
cast_df = pd.DataFrame({'movie_id': id_movie,
                     'person_id': id_star})

In [27]:
cast = Table(
    'cast', meta,
    Column('id', Integer, primary_key = True, autoincrement='auto'),
    Column('movie_id', String(8), ForeignKey(movies.c.id)),
    Column('person_id', String(8), ForeignKey(person.c.id)),
    extend_existing=True
)
meta.create_all(engine)

In [29]:
conn1 = pymysql.connect(host='localhost', user='root', passwd='1051047595Ma', db='schema_test')

# Insert desired columns from dataframe into movies table
with conn1:
    cursor = conn1.cursor()
    for index, row in cast_df[['movie_id', 'person_id']].iterrows():
        sql = "INSERT INTO cast (movie_id, person_id) VALUES (%s, %s)"
        cursor.execute(sql, tuple(row))
    conn1.commit()
    
print("DataFrame inserted into MySQL")

DataFrame inserted into MySQL


Crew

In [8]:
id_movie = full_ds['movie_id']
id_movie = np.concatenate((id_movie, full_wr['movie_id']))
id_direc = full_ds['id_director']
id_writer = full_wr['ids_writer']
role = np.array(['Director']*len(id_direc))
role = np.concatenate((role, np.array(['Writer']*len(id_writer))))
crew_df = pd.DataFrame({
    'movie_id': id_movie,
    'person_id': np.concatenate((id_direc, id_writer)),
    'role': role
})

In [23]:
crew_df.describe()

,movie_id,person_id,role
count,921,921,921
unique,250,511,2
top,09362722,0634240,Writer
freq,12,16,640


In [9]:
crew_df.drop_duplicates(['person_id'], inplace=True)
crew_df.describe()

,movie_id,person_id,role
count,511,511,511
unique,221,511,2
top,00031381,0001104,Writer
freq,6,1,335


In [30]:
crew = Table(
    'crew', meta,
    Column('id', Integer, primary_key = True, autoincrement='auto'),
    Column('movie_id', String(8), ForeignKey(movies.c.id)),
    Column('person_id', String(8), ForeignKey(person.c.id)),
    Column('role', String(8))
)
meta.create_all(engine)

In [11]:
conn1 = pymysql.connect(host='localhost', user='root', passwd='1051047595Ma', db='schema_test')

# Insert desired columns from dataframe into movies table
with conn1:
    cursor = conn1.cursor()
    for index, row in crew_df[['movie_id', 'person_id', 'role']].iterrows():
        sql = "INSERT INTO crew (movie_id, person_id, role) VALUES (%s, %s, %s)"
        cursor.execute(sql, tuple(row))
    conn1.commit()
    
print("DataFrame inserted into MySQL")

DataFrame inserted into MySQL


Genres

In [15]:
id_movie = full_ds['movie_id']
genre = full_ds['genre']
genre_df = pd.DataFrame({'movie_id': id_movie,
                     'genre': genre})

In [5]:
genre_movie = Table(
    'genre_movie', meta,
    Column('id', Integer, primary_key = True, autoincrement='auto'),
    Column('movie_id', String(8), ForeignKey(movies.c.id)),
    Column('genre', String(16))
)
# meta.create_all(engine)

In [16]:
conn1 = pymysql.connect(host='localhost', user='root', passwd='1051047595Ma', db='schema_test')

# Insert desired columns from dataframe into movies table
with conn1:
    cursor = conn1.cursor()
    for index, row in genre_df[['movie_id', 'genre']].iterrows():
        sql = "INSERT INTO genre_movie (movie_id, genre) VALUES (%s, %s)"
        cursor.execute(sql, tuple(row))
    conn1.commit()
    
print("DataFrame inserted into MySQL")

DataFrame inserted into MySQL


In [8]:
dele = genre_movie.delete().where(genre_movie.c.genre == 'Fi')
conn.execute(dele)